In [1]:
! pip install transformers
! pip install datasets
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import f1_score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.6 MB/s 
     |████████████████████████████████| 6.6 MB 69.6 MB/s 
     |████████████████████████████████| 120 kB 98.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 9.2 MB/s 
     |████████████████████████████████| 115 kB 58.2 MB/s 
     |████████████████████████████████| 212 kB 73.0 MB/s 
     |████████████████████████████████| 127 kB 79.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:

from transformers import AutoTokenizer, AutoModel, AdamW
from collections import defaultdict
from transformers import get_linear_schedule_with_warmup

import time
# from torchmetrics import MeanSquaredError
import torch
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

In [3]:

import numpy as np

from sklearn.metrics import f1_score
import datasets
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from transformers import BertModel, BertTokenizer
from numpy import *
from torch import nn, optim

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/新建文件夹/ERAI Dataset_with_translation

Mounted at /content/drive
/content/drive/MyDrive/新建文件夹/ERAI Dataset_with_translation


In [5]:
df = pd.read_json('ERAI_Dataset_pairwise_train_with_translation.json')
df2 = pd.read_json('ERAI_Dataset_pairwise_test_with_translation.json')
df = df.dropna(axis = 0)

pd.set_option('display.max_rows', None)

df.reset_index(drop = True,inplace = True)

df['post1']=df['post1'].apply(lambda x:x.replace('\n',''))
df['post2']=df['post2'].apply(lambda x:x.replace('\n',''))

df['post1_en']=df['post1_en'].apply(lambda x:x.replace('\n',''))
df['post2_en']=df['post2_en'].apply(lambda x:x.replace('\n',''))


In [6]:
df_unsupervised = pd.read_json('ERAI_Dataset_unsupervised_with_translation.json')

In [7]:
PRETRAINED_MODEL_NAME = "bert-base-chinese"
# 获取预测模型所使用的tokenizer
# AutoTokenizer, AutoModel,


#"bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [8]:

def generate_input_data(post,mpp_all_value,ml_all_value,max_len):

 
  
  # all_value = pd.concat([mpp_all_value,ml_all_value],axis=1)
  
  
  full_text = []
  attention_mask = []
  
  for i in post:
    if len(i)>max_len:
      i = i[:max_len]
    
    i = tokenizer.tokenize(i)
    
    text = ['[CLS]']+i+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))
    
  all_pad_tokens = pad_sequences(full_text, maxlen=max_len, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len, padding='post', truncating='post')
  
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),  
      'mpp_value': torch.tensor(mpp_all_value.values, dtype=torch.float),
      
      'ml_value':torch.tensor(ml_all_value.values,dtype = torch.float)
  }

    

In [9]:
 class BertNerClassifier(nn.Module):

  def __init__(self, n_classes):
    super(BertNerClassifier, self).__init__()
    self.bert = AutoModel.from_pretrained("bert-base-chinese")
    self.drop = nn.Dropout(p=0.1)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask):
    output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask

    )[1]
    
    output = self.drop(output)
    # self.out(output)
    
    return self.out(output)

In [10]:

class DataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      
      self.mpp_value = data['mpp_value']
      self.ml_value = data['ml_value']
     

    def __len__(self):

        return len(self.features)

    def __getitem__(self, index):

        return self.features[index],self.mask[index],self.mpp_value[index],self.ml_value[index]

In [11]:
def generate_dataloader(post,mpp_all_value,ml_all_value,max_len,batch_size):
  data = generate_input_data(post,mpp_all_value,ml_all_value,max_len)
  data = DataSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  return dataloader


In [12]:
EPOCHS = 15


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
loss_fn = nn.MSELoss().to(device)


In [16]:
def test_prediction(model, 
  test_dataloader, 
  loss_fn, 
  optimizer,
  device,
  mpp_flag 
  ):
  
  
  test_pred = []
  real_mpp_label = []
  real_ml_label = []

  

  model.eval()
  with torch.no_grad():
    for data in test_dataloader:
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        
      )
      
      for i in outputs:
        test_pred.append(i.item())
 
  return test_pred


      


     

In [26]:

def train_and_eval(model, 
  train_loader, 
  loss_fn, 
  optimizer,  
  device,
  epoch,
  label_num,
  test_dataloader1,
  test_dataloader2,
  mpp_flag,
  test_mpp_label,
  test_ml_label

  ):
  model.train()
  current_accuracy_state = 0
  for epo in range(epoch):
    
    
    train_loss = []
    
    train_pred = []
    real_train_label  = []
    

    
    for data in train_loader:
      
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)

      if mpp_flag == True:
        targets = data[2].to(device)
      if mpp_flag == False:
        targets = data[3].to(device)
      

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        
      )
      

      

      loss = loss_fn(outputs, targets)
 
      train_loss.append(loss.item())
      
  
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    test_pred1= test_prediction(model, 
      test_dataloader1, 
      loss_fn, 
      optimizer,
      device,
      mpp_flag
      )
    test_pred2= test_prediction(model, 
      test_dataloader2, 
      loss_fn, 
      optimizer,
      device,
      mpp_flag
      )
    test_pred = []
    real_label = []
    if mpp_flag == True:
      for i,j in zip(test_pred1,test_pred2):
        if i>j:
          test_pred.append(1)
        else:
          test_pred.append(0)
    else:
      for i,j in zip(test_pred1,test_pred2):
        if i>j:
          test_pred.append(0)
        else:
          test_pred.append(1)



    final_train_loss = np.mean(train_loss)
    train_accuracy = accuracy_score(real_train_label,train_pred)
    if mpp_flag == True:
      test_accuracy = accuracy_score(test_mpp_label,test_pred)
    else:
      test_accuracy = accuracy_score(test_ml_label,test_pred)

    if mpp_flag == True:
      if test_accuracy > current_accuracy_state:
        current_accuracy_state =  test_accuracy
        torch.save(model.state_dict(), 'regression_single_value_mpp.bin')
    else:
      if test_accuracy > current_accuracy_state:
        current_accuracy_state =  test_accuracy
        torch.save(model.state_dict(), 'regression_single_value_ml.bin')

    if mpp_flag == True:

      print(
'Epoch [{}] Train Totol Loss: {:.4f}, Valid MPP ACCURACY: {:.4f}'
    .format(epo+1,final_train_loss,test_accuracy))
    else:
      print(
'Epoch [{}] Train Totol Loss: {:.4f}, Valid ML ACCURACY: {:.4f}'
    .format(epo+1,final_train_loss,test_accuracy))
    


In [ ]:

for i in [171,354,550,667,985]:

  df = shuffle(df,random_state=i)

  train = df[0:156]

  test = df[156:200]


  train_post1 = train['post1']
  train_post2 = train['post2']
  train_mmp1 = train['MPP1']
  train_mpp2 = train['MPP2']
  train_ml1 = train['ML1']
  train_ml2 = train['ML2']

  all_train_post = pd.concat([train_post1,train_post2],axis=0)
  all_train_mpp = pd.concat([train_mmp1,train_mpp2],axis=0)
  all_train_ml = pd.concat([train_ml1,train_ml2],axis=0)

  max_len = 512
  batch_size = 16
  train_dataloader = generate_dataloader(all_train_post,all_train_mpp,all_train_ml,max_len,batch_size)

  test_post1 = test['post1']
  test_post2 = test['post2']
  test_mmp1 = test['MPP1']
  test_mpp2 = test['MPP2']
  test_ml1 = test['ML1']
  test_ml2 = test['ML2']
  test_mpp_label = test['MPP_label']
  test_ml_label = test['ML_label']



  all_test_post = pd.concat([test_post1,test_post2],axis=0)
  all_test_mpp = pd.concat([test_mmp1,test_mpp2],axis=0)
  all_test_ml = pd.concat([test_ml1,test_ml2],axis=0)




  test_dataloader1 = generate_dataloader(test_post1,test_mmp1,test_ml1,max_len,batch_size)
  test_dataloader2 = generate_dataloader(test_post2,test_mpp2,test_ml2,max_len,batch_size)



  label_num = 1
  model_mpp= BertNerClassifier(label_num).to(device)
  model_ml= BertNerClassifier(label_num).to(device)

  optimizer_mpp = AdamW(model_mpp.parameters(), lr=2e-5, correct_bias=False)
  optimizer_ml = AdamW(model_ml.parameters(), lr=2e-5, correct_bias=False)
  mpp_flag = True
  train_and_eval(model_mpp, 
      train_dataloader, 
      loss_fn, 
      optimizer_mpp,  
      device,
      EPOCHS,
      
      label_num,
      test_dataloader1,
      test_dataloader2,
      mpp_flag,
      test_mpp_label,
      test_ml_label

    )
  mpp_flag = False
  train_and_eval(model_ml,
      train_dataloader, 
      loss_fn, 
      optimizer_ml,  
      device,
      EPOCHS,
      label_num,
      test_dataloader1,
      test_dataloader2,
      mpp_flag,
      test_mpp_label,
      test_ml_label
    )


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.pr

In [19]:
model_mpp.load_state_dict(torch.load('regression_single_value_mpp.bin', map_location=torch.device("cuda:0")))
model_ml.load_state_dict(torch.load('regression_single_value_mpp.bin', map_location=torch.device("cuda:0")))

<All keys matched successfully>

In [20]:

def generate_test_data(post,max_len):
 
  
  full_text = []
  attention_mask = []
  
  for i in post:
    if len(i)>max_len:
      i = i[:max_len]
    
    i = tokenizer.tokenize(i)
    
    text = ['[CLS]']+i+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))
    
  all_pad_tokens = pad_sequences(full_text, maxlen=max_len, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len, padding='post', truncating='post')
  
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),

  }


In [21]:

class testDataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      

    def __len__(self):

        return len(self.features)

    def __getitem__(self, index):

        return self.features[index],self.mask[index]

In [22]:
def generate_test_dataloader(post,max_len,batch_size):
  data = generate_test_data(post,max_len)
  data = testDataSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  return dataloader

In [23]:
test_post1 = df2['post1']
test_post2 = df2['post2']

max_len = 512
test_dataloader1 = generate_test_dataloader(test_post1,max_len,1)
test_dataloader2 = generate_test_dataloader(test_post2,max_len,1)

In [24]:
test_pred_mpp1 = test_prediction(model_mpp, 
      test_dataloader1, 
      loss_fn, 
      optimizer_mpp,
      device,
      mpp_flag =True
      )
test_pred_mpp2 = test_prediction(model_mpp, 
      test_dataloader1, 
      loss_fn, 
      optimizer_mpp,
      device,
      mpp_flag =True
      )
mpp_prediction = []
for i,j in zip(test_pred_mpp1,test_pred_mpp2):
  if i>j:
    mpp_prediction.append(1)
  else:
    mpp_prediction.append(0)
test_pred_ml1 = test_prediction(model_ml, 
      test_dataloader2, 
      loss_fn, 
      optimizer_ml,
      device,
      mpp_flag =False
    )
test_pred_ml2 = test_prediction(model_ml, 
      test_dataloader2, 
      loss_fn, 
      optimizer_ml,
      device,
      mpp_flag =False
    )
ml_prediction = []
for i,j in zip(test_pred_ml1,test_pred_ml2):
  if i>j:
    ml_prediction.append(0)
  else:
    ml_prediction.append(1)






In [25]:
df2['MPP_prediction'] = mpp_prediction
df2['ML_prediction'] = ml_prediction

In [ ]:
df2.to_json('ERAI_pairwise_[aiml]_[1].json')